In [3]:
import pandas as pd
from textblob import TextBlob
import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer
import numpy as np

/usr/local/lib/python3.6/dist-packages/nltk/twitter/__init__.py:20: UserWarning: The twython library has not been installed. Some functionality from the twitter package will not be available.
  warnings.warn("The twython library has not been installed. "


In [4]:
! pip install vaderSentiment

     |████████████████████████████████| 133kB 4.9MB/s 


In [11]:
nltk.download('vader_lexicon')

[nltk_data] Downloading package vader_lexicon to /root/nltk_data...


True

In [0]:
url = 'https://raw.githubusercontent.com/zfz/twitter_corpus/master/full-corpus.csv'
dF = pd.read_csv(url)

In [6]:
dF.head()

,Topic,Sentiment,TweetId,TweetDate,TweetText
0,apple,positive,126415614616154112,Tue Oct 18 21:53:25 +0000 2011,Now all @Apple has to do is get swype on the i...
1,apple,positive,126404574230740992,Tue Oct 18 21:09:33 +0000 2011,@Apple will be adding more carrier support to ...
2,apple,positive,126402758403305474,Tue Oct 18 21:02:20 +0000 2011,Hilarious @youtube video - guy does a duet wit...
3,apple,positive,126397179614068736,Tue Oct 18 20:40:10 +0000 2011,@RIM you made it too easy for me to switch to ...
4,apple,positive,126395626979196928,Tue Oct 18 20:34:00 +0000 2011,I just realized that the reason I got into twi...


In [0]:
def textblobSentiment(tweet):
  return(TextBlob(tweet).sentiment.polarity)

In [8]:
dF["TextBlob polarity"] = dF["TweetText"].apply(textblobSentiment)
dF.head()

,Topic,Sentiment,TweetId,TweetDate,TweetText,TextBlob polarity
0,apple,positive,126415614616154112,Tue Oct 18 21:53:25 +0000 2011,Now all @Apple has to do is get swype on the i...,0.000000
1,apple,positive,126404574230740992,Tue Oct 18 21:09:33 +0000 2011,@Apple will be adding more carrier support to ...,0.500000
2,apple,positive,126402758403305474,Tue Oct 18 21:02:20 +0000 2011,Hilarious @youtube video - guy does a duet wit...,0.393750
3,apple,positive,126397179614068736,Tue Oct 18 20:40:10 +0000 2011,@RIM you made it too easy for me to switch to ...,0.541667
4,apple,positive,126395626979196928,Tue Oct 18 20:34:00 +0000 2011,I just realized that the reason I got into twi...,0.200000


In [0]:
def sentimentNltk(tweet):
  ss = SentimentIntensityAnalyzer().polarity_scores(tweet)
  m = max(ss, key = ss.get)
  d = {}
  d[m] = ss[m]
  return d

In [12]:
dF["Sentiment Intensity Analyze"] = dF["TweetText"].apply(sentimentNltk)
dF.head()

,Topic,Sentiment,TweetId,TweetDate,TweetText,TextBlob polarity,Sentiment Intensity Analyze
0,apple,positive,126415614616154112,Tue Oct 18 21:53:25 +0000 2011,Now all @Apple has to do is get swype on the i...,0.000000,{'neu': 1.0}
1,apple,positive,126404574230740992,Tue Oct 18 21:09:33 +0000 2011,@Apple will be adding more carrier support to ...,0.500000,{'neu': 0.801}
2,apple,positive,126402758403305474,Tue Oct 18 21:02:20 +0000 2011,Hilarious @youtube video - guy does a duet wit...,0.393750,{'compound': 0.8858}
3,apple,positive,126397179614068736,Tue Oct 18 20:40:10 +0000 2011,@RIM you made it too easy for me to switch to ...,0.541667,{'neu': 0.814}
4,apple,positive,126395626979196928,Tue Oct 18 20:34:00 +0000 2011,I just realized that the reason I got into twi...,0.200000,{'neu': 0.791}


In [0]:
data = pd.read_json('http://snap.stanford.edu/data/amazon/productGraph/categoryFiles/reviews_Office_Products_5.json.gz',lines=True)

In [0]:
dF2 = pd.DataFrame(data)

In [16]:
dF2['TextBlob polarity']=dF2['reviewText'].apply(textblobSentiment)
dF2.head()

,reviewerID,asin,reviewerName,helpful,reviewText,overall,summary,unixReviewTime,reviewTime,TextBlob polarity
0,A32T2H8150OJLU,B00000JBLH,ARH,"[3, 4]","I bought my first HP12C in about 1984 or so, a...",5,"A solid performer, and long time friend",1094169600,"09 3, 2004",0.180000
1,A3MAFS04ZABRGO,B00000JBLH,"Let it Be ""Alan""","[7, 9]",WHY THIS BELATED REVIEW? I feel very obliged t...,5,"Price of GOLD is up, so don't bury the golden ...",1197676800,"12 15, 2007",0.170939
2,A1F1A0QQP2XVH5,B00000JBLH,Mark B,"[3, 3]",I have an HP 48GX that has been kicking for mo...,2,"Good functionality, but not durable like old HPs",1293840000,"01 1, 2011",0.204091
3,A49R5DBXXQDE5,B00000JBLH,R. D Johnson,"[7, 8]",I've started doing more finance stuff recently...,5,One of the last of an almost extinct species,1145404800,"04 19, 2006",0.214217
4,A2XRMQA6PJ5ZJ8,B00000JBLH,Roger J. Buffington,"[0, 0]",For simple calculations and discounted cash fl...,5,Still the best,1375574400,"08 4, 2013",0.350000


In [17]:
dF2["Sentiment Intensity Analyze"] = dF2["reviewText"].apply(sentimentNltk)
dF2.head()

,reviewerID,asin,reviewerName,helpful,reviewText,overall,summary,unixReviewTime,reviewTime,TextBlob polarity,Sentiment Intensity Analyze
0,A32T2H8150OJLU,B00000JBLH,ARH,"[3, 4]","I bought my first HP12C in about 1984 or so, a...",5,"A solid performer, and long time friend",1094169600,"09 3, 2004",0.180000,{'compound': 0.9831}
1,A3MAFS04ZABRGO,B00000JBLH,"Let it Be ""Alan""","[7, 9]",WHY THIS BELATED REVIEW? I feel very obliged t...,5,"Price of GOLD is up, so don't bury the golden ...",1197676800,"12 15, 2007",0.170939,{'compound': 0.9963}
2,A1F1A0QQP2XVH5,B00000JBLH,Mark B,"[3, 3]",I have an HP 48GX that has been kicking for mo...,2,"Good functionality, but not durable like old HPs",1293840000,"01 1, 2011",0.204091,{'neu': 0.72}
3,A49R5DBXXQDE5,B00000JBLH,R. D Johnson,"[7, 8]",I've started doing more finance stuff recently...,5,One of the last of an almost extinct species,1145404800,"04 19, 2006",0.214217,{'compound': 0.997}
4,A2XRMQA6PJ5ZJ8,B00000JBLH,Roger J. Buffington,"[0, 0]",For simple calculations and discounted cash fl...,5,Still the best,1375574400,"08 4, 2013",0.350000,{'compound': 0.9455}


In [0]:
def polarityClass(str):
  score = SentimentIntensityAnalyzer().polarity_scores(str)
  if score['compound'] <= -0.5:
    return 'negative'
  if score['compound'] >= 0.5:
    return 'positive'
  else:
    return 'neutral'

In [0]:
dF["Polarity Class"] = dF["TweetText"].apply(polarityClass)
dF.head()

In [0]:
dF2["Polarity Class"] = dF2["reviewText"].apply(polarityClass)
dF2.head()

In [0]:
from sklearn.metrics import confusion_matrix
m = confusion_matrix(dF["Sentiment"], dF["Polarity Class"])
m